#### Feature Engineering:

In this section we will perform feature engineering some of the things we will perfrom are listed below

1. Add in the log transformation of the numerical variables
2. One-hot encode the categorical variables
3. Remove the Colinear features from the dataset

In [3]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv("D:\Sujit\Energy_Star_Score of a building\preprocessed.csv")
data = data.rename(columns = {'ENERGY STAR Score': 'score'})

In [9]:
"""
Selecting only the numerical values and then we will iterate through all the numerical values .

then transform the values into the LOG of the features , we have already discueed why this done.

"""
numeric_subset = data.select_dtypes('number')

for col in numeric_subset.columns:
    if col == 'score':
        next
    else:
        numeric_subset['log_' + col] = np.log(numeric_subset[col])

D:\Installed\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Installed\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\Installed\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
"""

now seleting all the categorial values and then vectorize them as one hot encodings so that we represent them as numnerical
features 


"""
categorical_subset = data[['Borough', 'Largest Property Use Type']]

"""
here to cectorise we can use any vectorizer like Count vectorizer and all .

"""
categorical_subset = pd.get_dummies(categorical_subset)

# Join the two dataframes using concat
# Make sure to use axis = 1 to perform a column bind
features = pd.concat([numeric_subset, categorical_subset], axis = 1)

features.shape

(11746, 115)

In [29]:
"""

Utility function to remove all the collinear features from the dataset .

and this function has two inputs

"""

def collinear_features(x, threshold):
    y = x['score']
    x = x.drop(columns = ['score'])
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []
    
    
    
    
    """
    
    Looping over every comluns finding the colinearity .
    
    
    """
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            
            
            
            
    """
    will only keep the features that has higher threshold than 0.6, and the others will be droped 
    
    """
    if val >= threshold:
        drop_cols.append(col.values[0])
    """
    
    Now droping all the values that have a higer corelation among them slef as we have seen in the Corealltion metrix in the 
    Exploratory Data Analysis , as we want only to keep the values which has higher corelation with the target values. 
    
    """
    
    
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    x = x.drop(columns = ['Weather Normalized Site EUI (kBtu/ft²)','Water Use (All Water Sources) (kgal)','log_Water Use (All Water Sources) (kgal)',
                          'Largest Property Use Type - Gross Floor Area (ft²)'])

    x['score'] = y
               
    return x

In [13]:
"""
now we are removing all the fetaure with a thershold of Grater than 0.6 becasue when the features have high corealltion 
between them selfs then it no point in having them .

"""
features = collinear_features(features, 0.6)

## Split data into train and validation sets.

In [32]:
no_score = features[features['score'].isna()]
score = features[features['score'].notnull()]

print(no_score.shape)
print(score.shape)

(1858, 65)
(9461, 65)


In [ ]:
features = score.drop(columns='score')
targets = pd.DataFrame(score['score'])
features = features.replace({np.inf: np.nan, -np.inf: np.nan})

In [33]:

"""
we should consider the test and train split of data as 70 - 30 ratio or 80 -20 ratio for better Genaralization


if it was calssification then for spliting we can use the stratified split but regreesion so not required 
"""
X, X_test, y, y_test = train_test_split(features, targets, test_size = 0.3)

print(X.shape)
print(X_test.shape)
print(y.shape)
print(y_test.shape)

(6622, 64)
(2839, 64)
(6622, 1)
(2839, 1)


In [36]:
X.to_csv('training_data.csv', index = False)
X_test.to_csv('testing_data.csv', index = False)
y.to_csv('training_target.csv', index = False)
y_test.to_csv('testing_target.csv', index = False)